In [160]:
import os
os.chdir("../")

In [162]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [189]:
resultDir = 'D:\Program Files\learn2brancher\\results'
problem = 'indset' # choices=['setcover', 'cauctions', 'facilities', 'indset']
problemSizes = ['small', 'medium']
models = ['gcnn:Lodi','gcnn:GraphConv'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [191]:
# f1 = "cauctions_20210710-004536.csv"
# f2 = "cauctions_20210710-095340.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
## df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

df = pd.read_csv(f'{resultDir}/indset_20211130-202143.csv')

In [5]:
# f1 = "setcover_20210725-205924.csv"
# f2 = "setcover_20210726-163252.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
# df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

In [192]:
# eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
# eval_file = eval_files[-1]
# eval_file = f"{resultDir}/cauctions_20210710-004536.csv"

# df = pd.read_csv(eval_file)
# df_list = []
# [df[df['type'] == probSize] for probSize in problemSizes]

# df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = pd.concat([df[df['type'] == probSize] for probSize in problemSizes])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()

In [193]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [194]:
dfgcnns_gmean = df_gcnns.groupby(['type','policy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','policy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes        stime
type   policy                                   
medium gcnn:GraphConv  72556.484981  1990.515891
       gcnn:Lodi       17159.131698   530.233589
small  gcnn:GraphConv     58.070292     9.626898
       gcnn:Lodi         164.493876    19.656512

In [195]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'gcnn:Lodi')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [196]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   policy                            
medium gcnn:GraphConv  4.228447  3.754036
       gcnn:Lodi       1.000000  1.000000
small  gcnn:GraphConv  0.353024  0.489756
       gcnn:Lodi       1.000000  1.000000

# 2. Std variances (per instance)

In [197]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','policy','instance']).std() / df_gcnns.groupby(['type','policy','instance']).mean()

In [198]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','policy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   policy                            
medium gcnn:GraphConv  0.317986  0.194000
       gcnn:Lodi       1.055033  1.015985
small  gcnn:GraphConv  0.807602  0.450945
       gcnn:Lodi       1.447822  0.927304

# 做一下 T-Test

# 计算Wins

In [199]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'gcnn:Lodi'),metric]
        for policy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,policy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,policy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,policy),(metric,)] = [t_stats, p_value]

In [200]:
ttest_res

nnodes               stime        
                        t_stats   p_value   t_stats p_value
type   policy                                              
medium gcnn:GraphConv  7.011514       0.0   7.06828     0.0
       gcnn:Lodi            0.0       1.0       0.0     1.0
small  gcnn:GraphConv -4.470817  0.000013 -7.284967     0.0
       gcnn:Lodi            0.0       1.0       0.0     1.0

In [201]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type    instance                                             seed
medium  data/instances/indset/transfer_1000_4/instance_1.lp  0       205.0
                                                             1       206.0
                                                             2       202.0
                                                             3       203.0
                                                             4       209.0
                                                                     ...  
small   data/instances/indset/transfer_500_4/instance_9.lp   0        80.0
                                                             1        81.0
                                                             2        87.0
                                                             3        83.0
                                                             4        84.0
Name: stime, Length: 200, dtype: float64

In [202]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [203]:
len(winner_idx)

200

In [204]:
dfgcnns_wins = df_gcnns.groupby(['type', 'policy'])['wins'].sum()
dfgcnns_wins

type    policy        
medium  gcnn:GraphConv    37
        gcnn:Lodi         63
small   gcnn:GraphConv    59
        gcnn:Lodi         41
Name: wins, dtype: int64

In [205]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

# 把Means、std 和 Wins合并起来

In [206]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std           wins
                         nnodes     stime    nnodes     stime wins
type   policy                                                     
medium gcnn:GraphConv  4.228447  3.754036  0.317986  0.194000   37
       gcnn:Lodi       1.000000  1.000000  1.055033  1.015985   63
small  gcnn:GraphConv  0.353024  0.489756  0.807602  0.450945   59
       gcnn:Lodi       1.000000  1.000000  1.447822  0.927304   41

In [207]:
# output_idx = pd.MultiIndex.from_product((dfgcnns_gmean_norm.index.get_level_values('type').unique(), sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx = pd.MultiIndex.from_product((problemSizes, models), names=['type', 'policy'])
output_idx

MultiIndex([( 'small',      'gcnn:Lodi'),
            ( 'small', 'gcnn:GraphConv'),
            ('medium',      'gcnn:Lodi'),
            ('medium', 'gcnn:GraphConv')],
           names=['type', 'policy'])

In [208]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [209]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   policy                                          
small  gcnn:Lodi         NaN   NaN  NaN             NaN
       gcnn:GraphConv    NaN   NaN  NaN             NaN
medium gcnn:Lodi         NaN   NaN  NaN             NaN
       gcnn:GraphConv    NaN   NaN  NaN             NaN

In [210]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [211]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [212]:
out_df1

nnodes              stime  wins  \
type   policy                                                       
small  gcnn:Lodi       1.0000r ± 144.78%   1.0000r ± 92.73%    41   
       gcnn:GraphConv   0.3530r ± 80.76%   0.4898r ± 45.09%    59   
medium gcnn:Lodi       1.0000r ± 105.50%  1.0000r ± 101.60%    63   
       gcnn:GraphConv   4.2284r ± 31.80%   3.7540r ± 19.40%    37   

                      t-stats (p-val)  
type   policy                          
small  gcnn:Lodi         0.00(1.0000)  
       gcnn:GraphConv   -7.28(0.0000)  
medium gcnn:Lodi         0.00(1.0000)  
       gcnn:GraphConv    7.07(0.0000)

In [27]:
out_df1.to_excel('evaluation_cauctions.xlsx')

In [122]:
out_df1.index.get_level_values(0)

Index(['small', 'small', 'medium', 'medium'], dtype='object', name='type')

In [123]:
pd.MultiIndex.from_product((problemSizes, ['uniform5']))

MultiIndex([( 'small', 'uniform5'),
            ('medium', 'uniform5')],
           )

In [62]:
out_df = pd.DataFrame()
for samp in sampling_Strategies:
    s = pd.Series()
    for probSize in problemSizes:
        s = s.append(out_df1.loc[(probSize, samp)])
    out_df[samp] = s

<ipython-input-62-049bb73f2ee1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series()


In [61]:
out_df

,uniform5,depthK,depthK2


In [58]:
out_df1.loc[('small', 'uniform5')].append(out_df1.loc[('medium', 'uniform5')])

nnodes             1.0000r ± 15.82%
stime               1.0000r ± 9.12%
wins                             39
t-stats (p-val)        0.00(1.0000)
nnodes             1.0000r ± 17.13%
stime              1.0000r ± 13.56%
wins                             22
t-stats (p-val)        0.00(1.0000)
dtype: object